In [2]:
import numpy as np
import pandas as pd
import random
from functools import reduce
import time
import torch
import openpyxl
import requests
from typing import Literal
import re
from enum import Enum
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Util

In [3]:
def convert_to_space(heading_list):
    cleaned_list = []
    for heading in heading_list:
        cleaned_str = heading.replace("\xa0", " ")
        cleaned_list.append(cleaned_str)
    return cleaned_list

## CSV Data

In [4]:
players = pd.read_csv('data/atp_players.csv')
players

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554
...,...,...,...,...,...,...,...,...
65014,212913,Pietro,Ricci,U,NaN,ITA,NaN,NaN
65015,212914,Corey,Craig,U,NaN,USA,NaN,NaN
65016,212915,Aleksandar,Ljubojevic,U,NaN,SRB,NaN,NaN
65017,212916,Marko,Milosavljevic,U,NaN,SRB,NaN,NaN


In [5]:
atp_singles = 'data/atp_singles/atp_matches_'

In [6]:
atp_2024 = pd.read_csv(f'{atp_singles}2024.csv')
atp_2024['tourney_date'] = pd.to_datetime(atp_2024['tourney_date'], format='%Y%m%d')
print(atp_2024.columns)

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')


In [7]:
atp_2024.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2024-0339,Brisbane,Hard,32,A,2024-01-01,300,105777,2,NaN,...,58.0,44.0,16.0,11.0,8.0,9.0,14.0,2570.0,8.0,3660.0
1,2024-0339,Brisbane,Hard,32,A,2024-01-01,299,208029,1,NaN,...,35.0,31.0,10.0,11.0,5.0,7.0,8.0,3660.0,39.0,1122.0
2,2024-0339,Brisbane,Hard,32,A,2024-01-01,298,105777,2,NaN,...,39.0,24.0,14.0,10.0,5.0,7.0,14.0,2570.0,55.0,902.0
3,2024-0339,Brisbane,Hard,32,A,2024-01-01,297,208029,1,NaN,...,51.0,31.0,16.0,10.0,3.0,5.0,8.0,3660.0,116.0,573.0
4,2024-0339,Brisbane,Hard,32,A,2024-01-01,296,126128,NaN,NaN,...,37.0,27.0,16.0,10.0,5.0,8.0,39.0,1122.0,44.0,1021.0


In [8]:
atp_2024['score']

0        7-6(5) 6-4
1        6-4 7-6(0)
2           6-3 7-5
3        6-2 7-6(6)
4        7-6(4) 6-2
           ...     
1412        6-1 6-0
1413        6-3 6-4
1414    4-6 6-3 4-0
1415    6-4 3-6 6-3
1416        6-2 6-2
Name: score, Length: 1417, dtype: object

In [9]:
### Score Reader
def score_reader(score):
    w_games_won = 0
    l_games_won = 0
    w_tiebreaks_won = 0
    l_tiebreaks_won = 0
    tiebreaks = 0
    tiebreak_points = []
    num_sets = 1

    i = 1
    for num in score[1:-1]:
        if num == '-':
            w_games_won += int(score[i-1])
            l_games_won += int(score[i+1])
        if num == '(':
            tiebreaks += 1
            if score[i-1] == '7':
                loser = 'l'
                w_tiebreaks_won += 1
            else:
                loser = 'w'
                l_tiebreaks_won += 1

            tiebreak_points.append(f"{loser} {score[i+1]}")
        if num == ' ':
            num_sets += 1
        i += 1
    total_games = w_games_won + l_games_won
    return w_games_won, l_games_won, total_games, tiebreaks, w_tiebreaks_won, l_tiebreaks_won, tiebreak_points, num_sets

In [10]:
def get_player_name(player_id):
    player = players[players['player_id'] == player_id]
    first_name = player['name_first'].iloc[0]
    last_name = player['name_last'].iloc[0]
    return first_name, last_name

In [11]:
def get_all_player_names(player_ids):
    first_names = pd.Series([players.loc[players['player_id'] == pid, 'name_first'].values[0] if not players.loc[players['player_id'] == pid, 'name_first'].empty else None for pid in player_ids])
    last_names = pd.Series([players.loc[players['player_id'] == pid, 'name_last'].values[0] if not players.loc[players['player_id'] == pid, 'name_last'].empty else None for pid in player_ids])
    
    return first_names, last_names

In [12]:
### Convert to Percentages
def get_percentages(orig_df):
    percentages = {}
    
    percentages['tourney_id'] = orig_df['tourney_id']
    percentages['tourney_name'] = orig_df['tourney_name']
    percentages['surface'] = orig_df['surface']
    percentages['draw_size'] = orig_df['draw_size']
    percentages['tourney_date'] = orig_df['tourney_date']
    percentages['match_num'] = orig_df['match_num']
    percentages['score'] = orig_df['score']
    percentages['winner_id'] = orig_df['winner_id']
    winner_first, winner_last = get_all_player_names(orig_df['winner_id'])
    percentages['winner_name'] = winner_first + ' ' + winner_last
    percentages['winner_seed'] = orig_df['winner_seed']
    percentages['winner_hand'] = orig_df['winner_hand']
    percentages['winner_ht'] = orig_df['winner_ht']
    percentages['winner_age'] = orig_df['winner_age']
    percentages['winner_rank'] = orig_df['winner_rank']
    percentages['winner_rank_points'] = orig_df['winner_rank_points']
    percentages['loser_id'] = orig_df['loser_id']
    loser_first, loser_last = get_all_player_names(orig_df['loser_id'])
    percentages['loser_name'] = loser_first + ' ' + loser_last
    percentages['loser_seed'] = orig_df['loser_seed']
    percentages['loser_hand'] = orig_df['loser_hand']
    percentages['loser_ht'] = orig_df['loser_ht']
    percentages['loser_age'] = orig_df['loser_age']
    percentages['loser_rank'] = orig_df['loser_rank']
    percentages['loser_rank_points'] = orig_df['loser_rank_points']

    percentages['w_ace_perc'] = orig_df['w_ace'] / orig_df['w_svpt']
    percentages['w_df_perc'] = orig_df['w_df'] / (orig_df['w_svpt'] - orig_df['w_1stIn'])
    percentages['w_first_in'] = orig_df['w_1stIn'] / orig_df['w_svpt']
    percentages['w_first_win_perc'] = orig_df['w_1stWon'] / orig_df['w_1stIn']
    percentages['w_second_in'] = 1 - percentages['w_df_perc']
    percentages['w_second_win_perc'] = orig_df['w_2ndWon'] / (orig_df['w_svpt'] - orig_df['w_1stIn'])
    percentages['w_bp_hold_perc'] = orig_df['w_bpSaved'] / orig_df['w_bpFaced']
    percentages['w_bp_win_perc'] = 1 - (orig_df['l_bpSaved'] / orig_df['l_bpFaced'])
    percentages['w_first_return_win_perc'] = 1 - orig_df['l_1stWon'] / orig_df['l_1stIn']
    percentages['w_second_return_win_perc'] = 1 - orig_df['l_2ndWon'] / (orig_df['l_svpt'] - orig_df['l_1stIn'])
    percentages['w_return_win_perc'] = (orig_df['l_bpFaced'] - orig_df['l_bpSaved']) / orig_df['l_SvGms']

    percentages['l_ace_perc'] = orig_df['l_ace'] / orig_df['l_svpt']
    percentages['l_df_perc'] = orig_df['l_df'] / (orig_df['l_svpt'] - atp_2024['l_1stIn'])
    percentages['l_first_in'] = orig_df['l_1stIn'] / orig_df['l_svpt']
    percentages['l_first_win_perc'] = orig_df['l_1stWon'] / orig_df['l_1stIn']
    percentages['l_second_in'] = 1 - percentages['l_df_perc']
    percentages['l_second_win_perc'] = orig_df['l_2ndWon'] / (orig_df['l_svpt'] - orig_df['l_1stIn'])
    percentages['l_bp_hold_perc'] = orig_df['l_bpSaved'] / orig_df['l_bpFaced']
    percentages['l_bp_win_perc'] = 1 - percentages['w_bp_hold_perc']
    percentages['l_first_return_win_perc'] = 1 - orig_df['w_1stWon'] / orig_df['w_1stIn']
    percentages['l_second_return_win_perc'] = 1 - orig_df['w_2ndWon'] / (orig_df['w_svpt'] - orig_df['w_1stIn'])
    percentages['l_return_win_perc'] = (orig_df['w_bpFaced'] - orig_df['w_bpSaved']) / orig_df['w_SvGms']
    
    return pd.DataFrame(percentages)

In [13]:
percentages = get_percentages(atp_2024)

In [14]:
# Player Data Search
def search_player(first_name, last_name, df=percentages):
    player_id = players.loc[(players['name_first'] == first_name) & (players['name_last'] == last_name), 'player_id'].iloc[0]
    data = df[(df['winner_id'] == player_id) | (df['loser_id'] == player_id)]
    data = data.sort_values(by='tourney_date', ascending=False)
    return data

In [15]:
### Filter out the stats of the opposing player
def filter_results(df, player_name):
    player_stats = []

    for index, row in df.iterrows():
        if row['winner_name'] == player_name:
            player_stat = {
                'tourney_id': row['tourney_id'],
                'tourney_name': row['tourney_name'], 
                'surface': row['surface'], 
                'tourney_date': row['tourney_date'], 
                'player': row['winner_name'], 
                'opponent': row['loser_name'],
                'score' : row['score'],
                'result': 'win',
                'ace_percentage' : row['w_ace_perc'],
                'df_percentage': row['w_df_perc'],
                'first_in': row['w_first_in'],
                'first_win_perc': row['w_first_win_perc'],
                'second_in': row['w_second_in'],
                'second_win_perc': row['w_second_win_perc'],
                'bp_hold_perc': row['w_bp_hold_perc'],
                'bp_win_perc': row['w_bp_win_perc'],
                'opp_first_in': row['l_first_in'],
                'first_return_win_perc' : row['w_first_return_win_perc'],
                'second_return_win_perc' : row['w_second_return_win_perc'],
                'return_win_perc': row['w_return_win_perc'],
            }
        elif row['loser_name'] == player_name:
            player_stat = {
                'tourney_id': row['tourney_id'],
                'tourney_name': row['tourney_name'], 
                'surface': row['surface'], 
                'tourney_date': row['tourney_date'], 
                'player': row['loser_name'], 
                'opponent': row['winner_name'],
                'score' : row['score'],
                'result': 'loss',
                'ace_percentage' : row['l_ace_perc'],
                'df_percentage': row['l_df_perc'],
                'first_in': row['l_first_in'],
                'first_win_perc': row['l_first_win_perc'],
                'second_in': row['l_second_in'],
                'second_win_perc': row['l_second_win_perc'],
                'bp_hold_perc': row['l_bp_hold_perc'],
                'bp_win_perc': row['l_bp_win_perc'],
                'opp_first_in': row['w_first_in'],
                'first_return_win_perc' : row['l_first_return_win_perc'],
                'second_return_win_perc' : row['l_second_return_win_perc'],
                'return_win_perc': row['l_return_win_perc'],
            }
        player_stats.append(player_stat)

    player_stats_df = pd.DataFrame(player_stats)
    
    return player_stats_df


In [16]:
def get_tiebreak_win_percentage(filtered_player_df):
    scores = filtered_player_df['score']
    results = filtered_player_df['result']
    tiebreaks_total = 0
    tiebreaks_won = 0
    for score, result in zip(scores, results):
        print("SCORE: ", score)
        w_games_won, l_games_won, total_games, tiebreaks, w_tiebreaks_won, l_tiebreaks_won, tiebreak_points, num_sets = score_reader(score)
        if result == 'win':
            tiebreaks_won += w_tiebreaks_won
        if result == 'loss':
            tiebreaks_won += l_tiebreaks_won
        print(tiebreaks_won)
        tiebreaks_total += tiebreaks
    return tiebreaks_won / tiebreaks_total

In [17]:
first = "Arthur"
last = "Fils"
player_f = filter_results(search_player(first, last), f"{first} {last}")
player_f

,tourney_id,tourney_name,surface,tourney_date,player,opponent,score,result,ace_percentage,df_percentage,first_in,first_win_perc,second_in,second_win_perc,bp_hold_perc,bp_win_perc,opp_first_in,first_return_win_perc,second_return_win_perc,return_win_perc
0,2024-0416,Rome Masters,Clay,2024-05-06,Arthur Fils,Alexandre Muller,7-5 6-3,loss,0.152778,0.162162,0.486111,0.771429,0.837838,0.405405,0.428571,0.333333,0.557377,0.294118,0.333333,0.100000
1,2024-1536,Madrid Masters,Clay,2024-04-22,Arthur Fils,Daniel Altmaier,6-2 6-3,loss,0.047619,0.259259,0.571429,0.638889,0.740741,0.333333,0.555556,0.500000,0.840000,0.309524,0.375000,0.111111
2,2024-0425,Barcelona,Clay,2024-04-15,Arthur Fils,Daniel Altmaier,6-4 1-6 6-1,win,0.015385,0.000000,0.630769,0.780488,1.000000,0.500000,0.333333,0.600000,0.671233,0.367347,0.458333,0.250000
3,2024-0425,Barcelona,Clay,2024-04-15,Arthur Fils,Alex De Minaur,7-5 6-2,win,0.071429,0.045455,0.607143,0.705882,0.954545,0.545455,0.333333,0.500000,0.594203,0.390244,0.607143,0.500000
4,2024-0425,Barcelona,Clay,2024-04-15,Arthur Fils,Dusan Lajovic,6-4 3-6 6-2,loss,0.065217,0.131579,0.586957,0.685185,0.868421,0.394737,0.375000,0.500000,0.783133,0.430769,0.333333,0.230769
5,2024-0410,Monte Carlo Masters,Clay,2024-04-08,Arthur Fils,Yannick Hanfmann,6-0 6-2,win,0.000000,0.176471,0.595238,0.840000,0.823529,0.470588,0.750000,0.666667,0.651163,0.571429,0.800000,0.857143
6,2024-0410,Monte Carlo Masters,Clay,2024-04-08,Arthur Fils,Lorenzo Musetti,6-3 7-5,loss,0.016129,0.043478,0.629032,0.538462,0.956522,0.565217,0.500000,0.666667,0.500000,0.322581,0.419355,0.181818
7,2024-7290,Estoril,Clay,2024-04-01,Arthur Fils,Joao Sousa,7-5 6-4,win,0.093333,0.111111,0.640000,0.750000,0.888889,0.481481,0.750000,0.500000,0.562500,0.355556,0.485714,0.272727
8,2024-7290,Estoril,Clay,2024-04-01,Arthur Fils,Cristian Garin,2-6 6-4 6-4,loss,0.035294,0.105263,0.552941,0.617021,0.894737,0.605263,0.428571,0.571429,0.681818,0.383333,0.500000,0.285714
9,2024-0403,Miami Masters,Hard,2024-03-18,Arthur Fils,Matteo Arnaldi,6-3 6-4,loss,0.014085,0.034483,0.591549,0.714286,0.965517,0.413793,0.800000,NaN,0.591837,0.068966,0.300000,0.000000


In [18]:
print(player_f.columns)

Index(['tourney_id', 'tourney_name', 'surface', 'tourney_date', 'player',
       'opponent', 'score', 'result', 'ace_percentage', 'df_percentage',
       'first_in', 'first_win_perc', 'second_in', 'second_win_perc',
       'bp_hold_perc', 'bp_win_perc', 'opp_first_in', 'first_return_win_perc',
       'second_return_win_perc', 'return_win_perc'],
      dtype='object')


## Data Scraping

In [19]:
player_first = "Jannik"
player_last = "Sinner"
general_url = "https://www.tennisabstract.com/cgi-bin/player.cgi?p=JannikSinner"
winners_ue_url = f"https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=206173/{player_first}-{player_last}&table=winners-errors"

In [20]:
driver = webdriver.Chrome()
driver.get(general_url)
gen_html_content = driver.page_source
gen_soup = BeautifulSoup(gen_html_content, "html.parser")
gen_pretty = gen_soup.prettify()
driver.quit()

In [68]:

class PlayerDataScraper():
    table_options = Literal["recent_results_serve", "recent_results_return", "winners_ues", "serve_speed", "key_points", "key_games", "point_by_point", 
                            "charting_serve", "charting_return", "charting_rally", "charting_tactics"]
    
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        self.general_url = f"https://www.tennisabstract.com/cgi-bin/player.cgi?p={first_name}{last_name}"
        self.pid = self.get_pid()
        self.player_stats_url = self.PlayerStatUrls(self)
    
    def scrape_html(self, url):
        driver = webdriver.Chrome()
        driver.get(url)
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        driver.quit()
        return soup
    
    def get_pid(self):
        profile_html = self.scrape_html(url=self.general_url)
        key_games = profile_html.find(string="Key Games   ")
        key_games_url = str(key_games.parent)
        pid = re.search(r'p=(\d+)/', key_games_url).group(1)
        return str(pid)
    
    def get_url_tables(self, stat_url):
        stat_html = self.scrape_html(stat_url)
        stat_all_tables = stat_html.find_all("table")
        stat_table = stat_all_tables[-1]
        stat_headers = convert_to_space([th.text.strip() for th in stat_table.find_all("th")])
        stat_rows = []
        for tr in stat_table.find_all("tr"):
            cells = [td.text.strip() for td in tr.find_all("td")]
            if cells:
                stat_rows.append(cells)
                
        return stat_rows, stat_headers
    
    def get_surface_speed(self):
        serve_url = "https://tennisabstract.com/reports/atp_surface_speed.html"
        surface_rows, surface_headers = self.get_url_tables(serve_url)
        surface_cols = [f"{col}_{i}" if surface_headers.count(col) > 1 else col for i, col in enumerate(surface_headers)]
        surface_df = pd.DataFrame(surface_rows, columns=surface_cols)
        surface_df = surface_df.dropna()
        surface_df = surface_df.replace(["", " ", None], np.nan)
        return surface_df
    
    def get_table_df(self, table_name: table_options):
        ### Gonna need to edit this so that it gets basic return stats from the recent matches table
        def add_duplicate_suffix(series):
            counts = series.value_counts()
            return series.where(counts == 1, series + '_' + series.groupby(series).cumcount().add(1).astype(str))
        
        stat_url = self.player_stats_url.get_url(table_name)
        stat_html = self.scrape_html(stat_url)
        stat_all_tables = stat_html.find_all("table")
        stat_table = stat_all_tables[-1]
        stat_headers = convert_to_space([th.text.strip() for th in stat_table.find_all("th")])
        stat_rows = []
        for tr in stat_table.find_all("tr"):
            cells = [td.text.strip() for td in tr.find_all("td")]
            if cells:
                stat_rows.append(cells)
        if table_name == 'charting_serve':
            stat_headers[14] = '2nd ' + stat_headers[14]
            stat_headers[15] = '2nd ' + stat_headers[15]
        
        stat_columns = [f"{col}_{i}" if stat_headers.count(col) > 1 else col for i, col in enumerate(stat_headers)]

        stat_df = pd.DataFrame(stat_rows, columns=stat_columns)
        if table_name == 'recent_results_serve':
            stat_df = stat_df.drop("More", axis=1)
            stat_df = stat_df.replace(["", " ", None], np.nan)
            stat_df = stat_df.dropna()
            match = match = stat_df.apply(lambda row: f"{row['Date'][-4:]} {row['Tournament']} {row['Rd']}".replace("Masters ", ""), axis=1)
            stat_df.insert(0, 'Match', match)
            spw = stat_df.apply(lambda row: f"{((float(row['1stIn'][:-1])) * float(row['1st%'][:-1]) + (100 - float(row['1stIn'][:-1])) * float(row['2nd%'][:-1])) * 0.01}%", axis=1)
            stat_df.insert(len(stat_df.columns) - 2, 'SPW', spw)
            # Write function to pull player name and then categorize them into a playstyle either by statistics or external api
            # Write function to pull opponent player name and add as a col
            stat_df['Match'] = add_duplicate_suffix(stat_df['Match'])
            return stat_df
        if table_name == 'recent_results_return':
            stat_df = stat_df.drop("More", axis=1)
            stat_df = stat_df.replace(["", " ", None], np.nan)
            stat_df = stat_df.dropna()
            match = match = stat_df.apply(lambda row: f"{row['Date'][-4:]} {row['Tournament']} {row['Rd']}".replace("Masters ", ""), axis=1)
            stat_df.insert(0, 'Match', match)
            stat_df['Match'] = add_duplicate_suffix(stat_df['Match'])
            return stat_df
        
        stat_df = stat_df.dropna()
        stat_df['Match'] = add_duplicate_suffix(stat_df['Match'])
        stat_df['Match'] = stat_df['Match'].str.replace('\xa0', ' ', regex=False)
        return stat_df
    
    def get_all_tables(self, delay=5):
        all_tables = {}
        for table_option in self.table_options.__args__:
            try:
                table_df = self.get_table_df(table_option)
                all_tables[table_option] = table_df
                time.sleep(delay)
            except Exception as e:
                print(f"Error retrieving {table_option}: {e}")
        return all_tables
    
    def get_recent_results(self, delay=5):
        recent_results = {}
        for table_option in ["recent_results_serve", "recent_results_return"]:
            try:
                table_df = self.get_table_df(table_option)
                recent_results[table_option] = table_df
                time.sleep(delay)
            except Exception as e:
                print(f"Error retrieving {table_option}: {e}")
        recent_results_serve = recent_results["recent_results_serve"]
        recent_results_return = recent_results["recent_results_return"]
        merged_df = recent_results_serve[["Match", "Date", "Surface", "vRk", "A%", "DF%", "1stIn", "1st%", "2nd%", "SPW"]].merge(
            recent_results_return[["Match", "vA%", "v1st%", "v2nd%", "RPW"]], on="Match", how="outer")
        merged_df["Date"] = merged_df["Date"].str.replace(r"[^\x00-\x7F]+", "-", regex=True)  # Normalize hyphens
        merged_df["Date"] = pd.to_datetime(merged_df["Date"], format="%d-%b-%Y")
        return merged_df.sort_values(by="Date", ascending=False)
    
    def get_merged_data(self, all_df_dict):
        recent_results_serve = all_df_dict["recent_results_serve"]
        recent_results_return = all_df_dict["recent_results_return"]
        merged_df = recent_results_serve[["Match", "Date", "Surface", "vRk", "A%", "DF%", "1stIn", "1st%", "2nd%", "SPW"]] # Add player playstyle col, add opponent name col
        merged_df = merged_df.merge(recent_results_return[["Match", "vA%", "v1st%", "v2nd%", "RPW"]], on="Match", how="outer")
        merged_df["Date"] = merged_df["Date"].str.replace(r"[^\x00-\x7F]+", "-", regex=True)  # Normalize hyphens
        merged_df["Date"] = pd.to_datetime(merged_df["Date"], format="%d-%b-%Y")
        
        table_to_cols = {
            "winners_ues": ["Match", "Ratio", "Wnr/Pt", "UFE/Pt", "RallyRatio", "Rally Wnr/Pt", "Rally UFE/Pt", "vs Ratio", "vs Wnr/Pt", "vs UFE/Pt"],
            "serve_speed": ["Match", "Avg Speed", "1st Avg", "2nd Avg"],
            "key_points": ["Match", "TB SPW", "TB RPW"],
            "key_games": ["Match", "BP Games", "BP Conv/BPG", "BPF Games", "Hold/BPFG", ],
            "point_by_point": ["Match"],
            "charting_serve": ["Match", "SvImpact_5", "Unret%", "<=3 W%_3", "RiP W%_4", "1st: Unret%", "<=3 W%_7", "RiP W%_8", "2nd: Unret%", "2nd <=3 W%", "2nd RiP W%"],
            "charting_return": ["Match", "RiP%", "RiP W%_3", "1st: RiP%", "RiP W%_9", "2nd: RiP%", "RiP W%_14"],
            "charting_rally": ["Match", "1-3 W%", "4-6 W%", "7-9 W%", "10+ W%"],
            "charting_tactics": ["Match", "Net Freq", "Net W%", "FH: Wnr%", "DTL Wnr%_7", "IO Wnr%", "BH: Wnr%", "DTL Wnr%_10"],
        }
        
        for table, df in all_df_dict.items():
            if table == "recent_results_serve":
                continue
            if table == "recent_results_return":
                continue
            selected_rows = df[table_to_cols[table]]
            merged_df = pd.merge(merged_df, selected_rows, on="Match", how="outer")
        
        return merged_df
    
    class PlayerStatUrls:
        TABLES = {
            "recent_results_serve": "recent_results_serve",
            "recent_results_return" : "recent_results_return",
            "winners_ues": "winners-errors",
            "serve_speed": "serve-speed",
            "key_points": "pbp-points",
            "key_games": "pbp-games",
            "point_by_point": "pbp-stats",
            "charting_serve": "mcp-serve",
            "charting_return": "mcp-return",
            "charting_rally": "mcp-rally",
            "charting_tactics": "mcp-tactics",
        }

        def __init__(self, scraper):
            self.scraper = scraper

        def get_url(self, table_name):
            if table_name not in self.TABLES:
                raise ValueError(f"Unknown table name: {table_name}")
            table = self.TABLES[table_name]
            if table == "recent_results_serve":
                return f"https://www.tennisabstract.com/cgi-bin/player-classic.cgi?p={self.scraper.first_name}{self.scraper.last_name}"
            elif table == "recent_results_return":
                return f"https://www.tennisabstract.com/cgi-bin/player-classic.cgi?p={self.scraper.first_name}{self.scraper.last_name}&f=r1"
            return f"https://www.tennisabstract.com/cgi-bin/player-more.cgi?p={self.scraper.pid}/{self.scraper.first_name}-{self.scraper.last_name}&table={table}"

        def __getattr__(self, item):
            if item in self.TABLES:
                return self.get_url(item)
            raise AttributeError(f"'PlayerStats' object has no attribute '{item}'")

### Test Data

In [69]:
jansin = PlayerDataScraper("Jannik", "Sinner")

In [47]:
all_dfs = jansin.get_all_tables()

In [48]:
merged_df = jansin.get_merged_data(all_dfs)
merged_df = merged_df.sort_values(by="Date", ascending=False)
merged_df.head(5)

,Match,Date,Surface,vRk,A%,DF%,1stIn,1st%,2nd%,SPW,...,4-6 W%,7-9 W%,10+ W%,Net Freq,Net W%,FH: Wnr%,DTL Wnr%_7,IO Wnr%,BH: Wnr%,DTL Wnr%_10
346,2025 Australian Open SF_1,2025-01-13,Hard,20,9.0%,0.0%,57.3%,74.5%,63.2%,69.6749%,...,55.9%,57.7%,51.7%,6.8%,78.6%,10.3%,20.5%,0.0%,7.8%,21.1%
345,2025 Australian Open R64_1,2025-01-13,Hard,173,14.0%,5.0%,62.0%,77.4%,55.3%,69.00200000000001%,...,62.0%,46.2%,69.2%,8.0%,68.4%,25.2%,18.5%,25.0%,16.7%,53.8%
344,2025 Australian Open R32_1,2025-01-13,Hard,46,9.6%,2.4%,60.2%,82.0%,51.5%,69.861%,...,60.0%,31.3%,55.6%,12.1%,70.0%,13.7%,22.5%,14.3%,7.6%,38.5%
343,2025 Australian Open R16_1,2025-01-13,Hard,13,13.3%,5.7%,56.2%,83.1%,58.7%,72.4128%,...,68.2%,45.5%,80.0%,5.6%,91.7%,12.4%,18.8%,0.0%,8.8%,33.3%
342,2025 Australian Open R128_1,2025-01-13,Hard,36,6.9%,1.0%,57.4%,82.8%,69.8%,77.26199999999999%,...,61.5%,37.5%,50.0%,3.5%,71.4%,17.8%,31.3%,50.0%,9.1%,36.4%


In [30]:
num_rows_with_nans = merged_df.isna().any(axis=1).sum()
print(num_rows_with_nans)

335


In [49]:
merged_df_cols = merged_df.columns
merged_df_cols

Index(['Match', 'Date', 'Surface', 'vRk', 'A%', 'DF%', '1stIn', '1st%', '2nd%',
       'SPW', 'vA%', 'v1st%', 'v2nd%', 'RPW', 'Ratio', 'Wnr/Pt', 'UFE/Pt',
       'RallyRatio', 'Rally Wnr/Pt', 'Rally UFE/Pt', 'vs Ratio', 'vs Wnr/Pt',
       'vs UFE/Pt', 'Avg Speed', '1st Avg', '2nd Avg', 'TB SPW', 'TB RPW',
       'BP Games', 'BP Conv/BPG', 'BPF Games', 'Hold/BPFG', 'SvImpact_5',
       'Unret%', '<=3 W%_3', 'RiP W%_4', '1st: Unret%', '<=3 W%_7', 'RiP W%_8',
       '2nd: Unret%', '2nd <=3 W%', '2nd RiP W%', 'RiP%', 'RiP W%_3',
       '1st: RiP%', 'RiP W%_9', '2nd: RiP%', 'RiP W%_14', '1-3 W%', '4-6 W%',
       '7-9 W%', '10+ W%', 'Net Freq', 'Net W%', 'FH: Wnr%', 'DTL Wnr%_7',
       'IO Wnr%', 'BH: Wnr%', 'DTL Wnr%_10'],
      dtype='object')

In [70]:
recent_results_df = jansin.get_recent_results()

In [73]:
recent_results_df

,Match,Date,Surface,vRk,A%,DF%,1stIn,1st%,2nd%,SPW,vA%,v1st%,v2nd%,RPW
62,2025 Australian Open SF_1,2025-01-13,Hard,20,9.0%,0.0%,57.3%,74.5%,63.2%,69.6749%,5.9%,42.9%,54.2%,47.5%
61,2025 Australian Open R64_1,2025-01-13,Hard,173,14.0%,5.0%,62.0%,77.4%,55.3%,69.00200000000001%,9.4%,28.9%,65.5%,43.5%
60,2025 Australian Open R32_1,2025-01-13,Hard,46,9.6%,2.4%,60.2%,82.0%,51.5%,69.861%,2.4%,33.3%,64.7%,46.3%
59,2025 Australian Open R16_1,2025-01-13,Hard,13,13.3%,5.7%,56.2%,83.1%,58.7%,72.4128%,4.5%,33.8%,50.0%,40.0%
58,2025 Australian Open R128_1,2025-01-13,Hard,36,6.9%,1.0%,57.4%,82.8%,69.8%,77.26199999999999%,13.3%,26.8%,51.9%,33.7%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,2024 Miami SF_1,2024-03-18,Hard,4,14.9%,2.1%,63.8%,80.0%,58.8%,72.3256%,6.0%,46.9%,61.1%,52.0%
27,2024 Miami R64_1,2024-03-18,Hard,148,5.8%,5.8%,69.2%,91.7%,43.8%,76.94680000000001%,13.7%,33.3%,38.9%,35.3%
26,2024 Miami R32_1,2024-03-18,Hard,26,20.0%,1.3%,68.0%,90.2%,66.7%,82.68%,9.8%,33.8%,53.1%,39.3%
25,2024 Miami R16_1,2024-03-18,Hard,66,3.7%,3.7%,61.1%,75.8%,71.4%,74.08840000000001%,7.0%,35.0%,58.1%,45.1%


In [74]:
surface_data = jansin.get_surface_speed()
surface_data

,Date,Tournament,Surface,Ace%,Surface Speed
0,2024-10-21,Basel,Hard,16.0%,1.53
1,2024-09-18,Chengdu,Hard,13.6%,1.43
2,2024-10-14,Antwerp,Hard,11.5%,1.36
3,2024-11-11,Tour Finals,Hard,13.4%,1.36
4,2024-12-18,Next Gen Finals,Hard,13.2%,1.33
...,...,...,...,...,...
65,2024-04-08,Monte Carlo Masters,Clay,6.1%,0.61
66,2024-04-15,Barcelona,Clay,4.9%,0.59
67,2024-04-15,Bucharest,Clay,4.8%,0.58
68,2024-07-22,Umag,Clay,5.6%,0.55


In [87]:
print(surface_data.loc[surface_data["Tournament"] == "Indian Wells Masters"])

          Date            Tournament Surface  Ace% Surface Speed
50  2024-03-04  Indian Wells Masters    Hard  8.6%          0.84


#### All Tables

In [543]:
rr = jansin.get_table_df("recent_results")
rr.head(20)

,Match,Date,Tournament,Surface,Rd,Rk,vRk,,Score,DR,A%,DF%,1stIn,1st%,2nd%,SPW,BPSvd,Time
1,2025 Australian Open F_1,13‑Jan‑2025,Australian Open,Hard,F,1,2,(1)Sinner d. (2)Alexander Zverev [GER],6-3 7-6(4) 6-3,1.52,6.3%,2.1%,60.0%,84.2%,63.2%,75.8%,0/0,2:42
2,2025 Australian Open SF_1,13‑Jan‑2025,Australian Open,Hard,SF,1,20,(1)Sinner d. (21)Ben Shelton [USA],7-6(2) 6-2 6-2,1.56,9.0%,0.0%,57.3%,74.5%,63.2%,69.6749%,5/7,2:36
3,2025 Australian Open QF_1,13‑Jan‑2025,Australian Open,Hard,QF,1,8,(1)Sinner d. (8)Alex De Minaur [AUS],6-3 6-2 6-1,2.80,3.4%,0.0%,64.4%,84.2%,81.0%,83.0608%,1/1,1:48
4,2025 Australian Open R16_1,13‑Jan‑2025,Australian Open,Hard,R16,1,13,(1)Sinner d. (13)Holger Rune [DEN],6-3 3-6 6-3 6-2,1.45,13.3%,5.7%,56.2%,83.1%,58.7%,72.4128%,4/5,3:13
5,2025 Australian Open R32_1,13‑Jan‑2025,Australian Open,Hard,R32,1,46,(1)Sinner d. Marcos Giron [USA],6-3 6-4 6-2,1.54,9.6%,2.4%,60.2%,82.0%,51.5%,69.861%,6/7,2:01
6,2025 Australian Open R64_1,13‑Jan‑2025,Australian Open,Hard,R64,1,173,(1)Sinner d. (WC)Tristan Schoolkate [AUS],4-6 6-4 6-1 6-3,1.40,14.0%,5.0%,62.0%,77.4%,55.3%,69.00200000000001%,0/1,2:46
7,2025 Australian Open R128_1,13‑Jan‑2025,Australian Open,Hard,R128,1,36,(1)Sinner d. Nicolas Jarry [CHI],7-6(2) 7-6(5) 6-1,1.48,6.9%,1.0%,57.4%,82.8%,69.8%,77.26199999999999%,2/2,2:40
8,2024 Davis Cup Finals F: ITA vs NED RR_1,24‑Nov‑2024,Davis Cup Finals F: ITA vs NED,Hard,RR,1,40,Sinner d. Tallon Griekspoor [NED],7-6(2) 6-2,1.40,25.0%,1.7%,73.3%,79.5%,56.3%,73.3056%,2/3,1:31
9,2024 Davis Cup Finals SF: ITA vs AUS RR_1,23‑Nov‑2024,Davis Cup Finals SF: ITA vs AUS,Hard,RR,1,9,Sinner d. Alex De Minaur [AUS],6-3 6-4,1.40,7.0%,0.0%,64.9%,81.1%,45.0%,68.4289%,0/1,1:28
10,2024 Davis Cup Finals QF: ITA vs ARG RR_1,21‑Nov‑2024,Davis Cup Finals QF: ITA vs ARG,Hard,RR,1,27,Sinner d. Sebastian Baez [ARG],6-2 6-1,1.74,12.1%,3.4%,53.4%,71.0%,63.0%,67.272%,4/4,1:12


In [503]:
winners = jansin.get_table_df("winners_ues")
winners.head(20)

,Match,Result,Winners,UFEs,Ratio,Wnr/Pt,UFE/Pt,RallyWinners,RallyUFEs,RallyRatio,Rally Wnr/Pt,Rally UFE/Pt,FH Wnr/Pt,BH Wnr/Pt,vs Ratio,vs Wnr/Pt,vs UFE/Pt
0,2025 Australian Open F_1,W vs Zverev,29,19,1.5,15.3%,10.0%,23,17,1.4,13.7%,10.1%,7.1%,6.0%,0.6,12.1%,21.6%
1,2025 Australian Open SF_1,W vs Shelton,23,27,0.9,11.1%,13.0%,15,27,0.6,8.1%,14.5%,4.8%,3.2%,0.4,12.6%,28.5%
2,2025 Australian Open QF_1,W vs De Minaur,26,23,1.1,19.0%,16.8%,23,23,1.0,17.6%,17.6%,10.7%,6.9%,0.3,6.6%,23.4%
3,2025 Australian Open R16_1,W vs Rune,32,33,1.0,14.9%,15.3%,18,27,0.7,9.7%,14.5%,5.4%,4.3%,0.5,13.5%,26.0%
4,2025 Australian Open R32_1,W vs Giron,33,39,0.8,20.4%,24.1%,25,37,0.7,16.9%,25.0%,12.2%,4.7%,0.3,6.2%,18.5%
5,2025 Australian Open R64_1,W vs Schoolkate,39,20,2.0,16.4%,8.4%,25,15,1.7,12.8%,7.7%,7.7%,5.1%,1.0,13.9%,13.4%
6,2025 Australian Open R128_1,W vs Jarry,19,21,0.9,9.5%,10.6%,12,20,0.6,6.9%,11.5%,5.7%,1.1%,0.7,19.6%,27.6%
7,2024 Davis Cup RR_1,W vs Griekspoor,23,13,1.8,19.3%,10.9%,7,12,0.6,7.5%,12.9%,6.5%,1.1%,0.9,16.8%,18.5%
8,2024 Davis Cup RR_2,W vs De Minaur,22,15,1.5,19.0%,12.9%,18,15,1.2,17.0%,14.2%,10.4%,6.6%,0.9,12.9%,13.8%
9,2024 Tour Finals F_1,W vs Fritz,33,17,1.9,28.2%,14.5%,19,17,1.1,20.7%,18.5%,14.1%,3.3%,1.7,27.4%,16.2%


In [526]:
merge = pd.merge(winners, rr, on="Match", how="outer")
merge.tail(20)

,Match,Result,Winners,UFEs,Ratio,Wnr/Pt,UFE/Pt,RallyWinners,RallyUFEs,RallyRatio,...,Score,DR,A%,DF%,1stIn,1st%,2nd%,SPW,BPSvd,Time
285,2024 US Open SF_1,W vs Draper,43,34,1.3,18.9%,15.0%,32,32,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,2024 Wimbledon QF_1,L vs Medvedev,61,45,1.4,18.8%,13.9%,44,41,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,2024 Wimbledon R128_1,W vs Hanfmann,47,30,1.6,19.6%,12.5%,31,27,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,2024 Wimbledon R16_1,W vs Shelton,28,29,1.0,15.1%,15.6%,21,29,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289,2024 Wimbledon R32_1,W vs Kecmanovic,35,15,2.3,25.0%,10.7%,24,13,1.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,2024 Wimbledon R64_1,W vs Berrettini,32,25,1.3,10.8%,8.4%,22,23,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291,2025 Australian Open F_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6-3 7-6(4) 6-3,1.52,6.3%,2.1%,60.0%,84.2%,63.2%,75.8%,0/0,2:42
292,2025 Australian Open QF_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6-3 6-2 6-1,2.80,3.4%,0.0%,64.4%,84.2%,81.0%,83.0608%,1/1,1:48
293,2025 Australian Open R128_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7-6(2) 7-6(5) 6-1,1.48,6.9%,1.0%,57.4%,82.8%,69.8%,77.26199999999999%,2/2,2:40
294,2025 Australian Open R16_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6-3 3-6 6-3 6-2,1.45,13.3%,5.7%,56.2%,83.1%,58.7%,72.4128%,4/5,3:13


In [497]:
serve = jansin.get_table_df("serve_speed")
serve.head(10)

,Match,Result,Avg Speed,1st Avg,1st StDev,1st T Avg,1st Wide Avg,Max 1st,Min 1st,2nd Avg,2nd StDev,2nd T Avg,2nd Wide Avg,Max 2nd,Min 2nd
0,2024 US Open F1,W vs Fritz,108.9,120.1,4.6,120.9,120.0,130,107,96.0,5.9,97.3,96.5,104,80
1,2024 US Open SF1,W vs Draper,106.9,116.9,5.8,117.8,115.7,127,102,88.2,6.8,83.7,89.5,100,74
2,2024 US Open QF1,W vs Medvedev,105.0,117.2,5.6,120.1,113.4,126,103,91.2,7.4,94.7,101.3,107,70
3,2024 US Open R161,W vs Paul,104.6,119.7,5.8,121.8,117.5,128,92,90.2,7.3,95.0,100.0,100,69
4,2024 US Open R321,W vs Oconnell,107.5,116.4,11.3,121.3,110.0,128,78,89.1,6.1,87.7,,96,74
5,2024 US Open R641,W vs Michelsen,107.2,118.8,6.2,120.4,117.8,130,96,90.6,8.4,95.0,83.0,101,64
6,2024 US Open R1281,W vs Mcdonald,106.4,118.3,8.8,122.1,115.1,131,86,89.6,7.3,91.8,83.5,103,74
7,2024 Wimbledon QF1,L vs Medvedev,112.7,120.3,4.4,122.0,118.9,128,108,97.8,9.8,104.2,103.3,115,80
8,2024 Wimbledon R161,W vs Shelton,112.6,119.0,4.6,121.5,115.4,129,108,97.6,5.7,99.6,91.8,106,83
9,2024 Wimbledon R321,W vs Kecmanovic,113.7,121.6,4.0,124.0,119.6,130,113,99.7,8.4,102.8,101.7,120,86


In [533]:
key_points = jansin.get_table_df("key_points")
key_points.head()

,Match,Result,GP Conv,BP Conv,SP Conv,MP Conv,BP Saved,SP Saved,MP Saved,TB SPW,TB RPW
0,2025 Australian Open F_1,W vs Zverev,70% (16/23),20% (2/10),3/3,1/1,- (0/0),0/0,0/0,67% (4/6),60% (3/5)
1,2025 Australian Open SF_1,W vs Shelton,67% (12/18),46% (6/13),3/4,1/1,71% (5/7),2/2,0/0,100% (5/5),50% (2/4)
2,2025 Australian Open QF_1,W vs De Minaur,92% (12/13),60% (6/10),3/4,1/2,100% (1/1),0/0,0/0,- (0/0),- (0/0)
3,2025 Australian Open R16_1,W vs Rune,74% (17/23),50% (4/8),3/3,1/1,80% (4/5),1/2,0/0,- (0/0),- (0/0)
4,2025 Australian Open R32_1,W vs Giron,87% (13/15),50% (5/10),3/4,1/1,86% (6/7),0/0,0/0,- (0/0),- (0/0)


In [449]:
key_games = jansin.get_table_df("key_games")
key_games.head()

,Match,Result,BP Games,BP Conv/BPG,BreakBack%,BPF Games,Hold/BPFG,Consol%,SvForSet,SvStaySet,SvForMatch,SvStayMatch
0,2025 Australian Open F,W vs Zverev,29% (4/14),50% (2/4),0/0,0% (0/16),- (0/0),2/2,2/2,2/2,1/1,0/0
1,2025 Australian Open SF,W vs Shelton,64% (9/14),67% (6/9),1/2,36% (5/14),60% (3/5),5/5,2/2,0/0,1/1,0/0
2,2025 Australian Open QF,W vs De Minaur,50% (6/12),100% (6/6),0/0,8% (1/12),100% (1/1),5/5,2/2,0/0,0/0,0/0
3,2025 Australian Open R16,W vs Rune,29% (5/17),80% (4/5),0/1,17% (3/18),67% (2/3),4/4,3/3,0/0,1/1,0/0
4,2025 Australian Open R32,W vs Giron,54% (7/13),71% (5/7),1/1,29% (4/14),75% (3/4),5/5,3/3,0/0,1/1,0/0


In [448]:
point_by_point = jansin.get_table_df("point_by_point")
point_by_point.head()

,Match,Result,BLR,DR+,EI,CBF,Deuce A%,Deuce SPW%,Ad A%,Ad SPW%,Deuce RPW%,Ad RPW%
0,2025 Australian Open F,W vs Zverev,1.06,1.61,37.2,2.1,10.2%,79.6%,2.2%,71.7%,27.5%,47.7%
1,2025 Australian Open SF,W vs Shelton,0.91,1.43,32.6,3.0,10.9%,71.7%,7.0%,67.4%,41.7%,53.4%
2,2025 Australian Open QF,W vs De Minaur,1.05,2.94,21.8,2.0,9.4%,84.4%,0.0%,81.5%,41.5%,54.1%
3,2025 Australian Open R16,W vs Rune,1.01,1.46,37.3,2.4,12.5%,73.2%,14.3%,71.4%,45.6%,34.0%
4,2025 Australian Open R32,W vs Giron,1.17,1.80,26.9,2.0,4.5%,72.7%,15.4%,66.7%,48.8%,43.6%


In [573]:
charting_serve = jansin.get_table_df("charting_serve")
charting_serve.head()

,Match,Result,Unret%,<=3 W%_3,RiP W%_4,SvImpact_5,1st: Unret%,<=3 W%_7,RiP W%_8,SvImpact_9,D Wide%,A Wide%,BP Wide%,2nd: Unret%,2nd <=3 W%,2nd RiP W%,2ndAgg
0,2025 Australian Open F_1,W vs Zverev,27.4%,36.8%,68.7%,33.5%,42.1%,56.1%,72.7%,52.3%,42.3%,54.8%,-,5.6%,8.3%,64.7%,-60
1,2025 Australian Open SF_1,W vs Shelton,30.3%,41.6%,56.5%,33.3%,42.0%,50.0%,55.2%,47.2%,61.5%,50.0%,50.0%,15.4%,30.8%,57.6%,-79
2,2025 Australian Open QF_1,W vs De Minaur,33.9%,49.2%,74.4%,43.1%,43.6%,59.0%,72.7%,57.4%,35.3%,54.5%,0.0%,15.0%,30.0%,76.5%,-79
3,2025 Australian Open R16_1,W vs Rune,35.2%,41.0%,62.9%,38.9%,49.2%,55.9%,66.7%,55.6%,41.4%,46.7%,33.3%,20.0%,25.0%,59.4%,73
4,2025 Australian Open R32_1,W vs Giron,31.3%,43.4%,57.4%,38.4%,44.9%,59.2%,66.7%,56.9%,47.6%,50.0%,33.3%,12.9%,22.6%,48.1%,13


In [574]:
charting_return = jansin.get_table_df("charting_return")
charting_return.head()

,Match,Result,RPW,RiP%,RiP W%_3,RetWnr%_4,FH/BH,RDI_6,Slice%_7,1st: RiP%,RiP W%_9,RetWnr%_10,RDI_11,Slice%_12,2nd: RiP%,RiP W%_14,RetWnr%_15,RDI_16,Slice%_17
0,2025 Australian Open F_1,W vs Zverev,34.7412%,66.3%,52.4%,0.0%,0/0,2.46,20.9%,61.5%,50.0%,0.0%,2.58,42.4%,82.1%,56.5%,0.0%,2.35,0.0%
1,2025 Australian Open SF_1,W vs Shelton,42.336%,72.0%,58.8%,1.7%,1/1,2.05,19.4%,68.6%,62.5%,0.0%,2.07,27.6%,88.1%,54.1%,4.8%,2.03,12.1%
2,2025 Australian Open QF_1,W vs De Minaur,46.1368%,80.8%,57.1%,2.6%,1/1,1.94,10.3%,75.0%,46.7%,2.5%,1.86,18.2%,89.2%,66.7%,2.7%,2.03,0.0%
3,2025 Australian Open R16_1,W vs Rune,36.371700000000004%,70.9%,51.3%,6.4%,3/4,2.10,11.3%,71.0%,46.9%,4.3%,2.17,20.0%,78.4%,58.6%,10.8%,2.03,3.1%
4,2025 Australian Open R32_1,W vs Giron,41.472%,76.8%,54.0%,2.4%,0/2,2.39,17.6%,68.9%,45.2%,2.2%,2.38,29.2%,97.0%,62.5%,3.0%,2.41,7.4%


In [575]:
charting_rally = jansin.get_table_df("charting_rally")
charting_rally.head()

,Match,Result,RallyLen,RLen-Serve,RLen-Return,1-3 W%,4-6 W%,7-9 W%,10+ W%,FH/GS,BH Slice%,FHP,FHP/100,BHP,BHP/100
0,2025 Australian Open F_1,W vs Zverev,5.0,5.9,4.1,51.6%,59.5%,50.0%,67.6%,46.7%,5.7%,20.5,11.5,13.0,6.5
1,2025 Australian Open SF_1,W vs Shelton,4.8,5.1,4.6,58.5%,55.9%,57.7%,51.7%,50.8%,7.1%,11.5,5.5,13.5,6.9
2,2025 Australian Open QF_1,W vs De Minaur,5.5,4.8,6.0,62.3%,69.0%,58.3%,59.3%,48.3%,4.7%,18.0,11.4,8.0,4.9
3,2025 Australian Open R16_1,W vs Rune,3.7,3.8,3.7,50.7%,68.2%,45.5%,80.0%,48.9%,4.8%,10.5,6.9,6.5,4.2
4,2025 Australian Open R32_1,W vs Giron,4.4,4.0,4.9,63.0%,60.0%,31.3%,55.6%,64.7%,8.2%,13.0,6.7,6.0,5.9


In [455]:
charting_tactics = jansin.get_table_df("charting_tactics")
charting_tactics.head()

,Match,Result,SnV Freq,SnV W%,Net Freq,Net W%,FH: Wnr%,DTL Wnr%_7,IO Wnr%,BH: Wnr%,DTL Wnr%_10,Drop: Freq,Wnr%,RallyAgg,ReturnAgg
0,2025 Australian Open F,W vs Zverev,0.0%,-,5.8%,90.9%,11.1%,31.3%,25.0%,8.3%,18.8%,1.5%,33.3%,-96,-74
1,2025 Australian Open SF,W vs Shelton,0.0%,-,6.8%,78.6%,10.3%,20.5%,0.0%,7.8%,21.1%,0.2%,0.0%,-84,-71
2,2025 Australian Open QF,W vs De Minaur,1.8%,0.0%,7.3%,70.0%,12.1%,13.5%,25.0%,6.8%,22.7%,0.6%,50.0%,-51,-29
3,2025 Australian Open R16,W vs Rune,0.0%,-,5.6%,91.7%,12.4%,18.8%,0.0%,8.8%,33.3%,0.3%,0.0%,-61,-8
4,2025 Australian Open R32,W vs Giron,4.0%,33.3%,12.1%,70.0%,13.7%,22.5%,14.3%,7.6%,38.5%,1.3%,25.0%,44,47


## Skill Scores

In [25]:
### Service Score
def service_score(first_name, last_name):
    stats = filter_results(search_player(first_name=first_name, last_name=last_name), f"{first_name} {last_name}")
    ace_perc = np.nanmean(stats['ace_percentage'])
    first_in = np.nanmean(stats['first_in'])
    first_win_perc = np.nanmean(stats['first_win_perc'])
    second_win_perc = np.nanmean(stats['second_win_perc'])
    df_perc = np.nanmean(stats['df_percentage'])
    bp_hold_perc = np.nanmean(stats['bp_hold_perc'])
    serve_win_perc = first_in * first_win_perc + (1 - first_in) * second_win_perc
    
    score = 0.25 * bp_hold_perc +  serve_win_perc + 1.5 * ace_perc - df_perc

    return score


In [26]:
### Return Score
def return_score(first_name, last_name):
    stats = filter_results(search_player(first_name=first_name, last_name=last_name), f"{first_name} {last_name}")
    bp_win_perc = np.nanmean(stats['bp_win_perc']) # 30-50
    first_return_win_perc = np.nanmean(stats['first_return_win_perc']) # 20-40
    second_return_win_perc = np.nanmean(stats['second_return_win_perc']) # 40-60
    return_win_perc = np.nanmean(stats['return_win_perc']) # 15 - 40
    
    score = return_win_perc + first_return_win_perc * 0.5 + second_return_win_perc * 0.33 + bp_win_perc * 0.5
    
    return score

In [27]:
### Pressure Score
def pressure_score(first_name, last_name):
    score = 0
    stats = filter_results(search_player(first_name=first_name, last_name=last_name), f"{first_name} {last_name}")
    bp_win_perc = np.average(stats['bp_win_perc'])
    bp_hold_perc = np.average(stats['bp_hold_perc'])
    tiebreak_win_perc = get_tiebreak_win_percentage(stats)
    print(tiebreak_win_perc)
    
    return score

## Model Experimentation

### Feature Gathering

make it so that when I am trying to get the serve data for p1, we pass in the other player's return and rally statistics as well as pressure etc  
 

In [ ]:
class GetPlayerFeatures():
    
    def __init__(self, p1_first, p1_last, p2_first, p2_last):
        p1_scraper = PlayerDataScraper(p1_first, p1_last)
        p2_scraper = PlayerDataScraper(p2_first, p2_last)
        
        self.p1_all_data = p1_scraper.get_all_tables()
        self.p2_all_data = p2_scraper.get_all_tables()
        
        self.p1_combined_df = pd.concat(self.p1_all_data, axis=1).dropna() ## this code doesn't work must call function I need to write still
        self.p2_combined_df = pd.concat(self.p2_all_data, axis=1).dropna()

    def get_serve_features():
        serve_features = []
        
        return serve_features
    
    def get_return_features():
        return_features = []
        
        return return_features
    
    def get_rally_features():
        rally_features = []
        
        return rally_features
    
    def get_pressure_features():
        pressure_features = []
        
        return pressure_features
        

### Serve Return Statistics

In [28]:
# Need to incorporate a way to weight by opponent ranking

In [145]:
class PlayerServeReturnStats():
    
    def __init__(self, first_name, last_name, num_weeks, current_tournament):
        player_scraper = PlayerDataScraper(first_name, last_name)
        self.recent_results = player_scraper.get_recent_results()
        self.num_weeks = num_weeks
        self.surface_data = player_scraper.get_surface_speed()
        self.current_tournament = current_tournament
        
    def gather_last_x_weeks(self, num_weeks=-1):
        abbreviated_data = self.recent_results[["Match", "Date", "vRk", "SPW", "RPW"]]
        abbreviated_data = abbreviated_data[~abbreviated_data["Match"].str.contains("Davis Cup", na=False)]
        if num_weeks == -1:
            return abbreviated_data
        most_recent_date = merged_df["Date"].max()
        cutoff_date = most_recent_date - pd.Timedelta(weeks=num_weeks)
        return abbreviated_data[abbreviated_data["Date"] >= cutoff_date]
    
    def normalize_data(self):
        abbreviated_data = self.gather_last_x_weeks(num_weeks=self.num_weeks)
        
        for index, row in abbreviated_data.iterrows():
            tournament = row["Match"]
            match = re.search(r"\d{4}\s+(.+?)\s+\S+$", tournament)
            if match:
                tournament_name = match.group(1)
            surface_speed = surface_data.loc[surface_data["Tournament"].str.contains(tournament_name, case=False), "Surface Speed"]
            
            if not surface_speed.empty:
                surface_speed = surface_speed.iloc[0]
            else:
                print("No Surface Speed Found for: ", tournament_name)

            spw_num = float(row["SPW"][:-1])
            rpw_num = float(row["RPW"][:-1])

            spw = spw_num / float(surface_speed)
            rpw = rpw_num * float(surface_speed)
            
            abbreviated_data.at[index, "SPW"] = spw
            abbreviated_data.at[index, "RPW"] = rpw
        return abbreviated_data
    
    def estimate_spw_rpw(self):
        normalized_data = self.normalize_data()
        
        spw = np.average(normalized_data["SPW"])
        rpw = np.average(normalized_data["RPW"])
        
        current_surface_speed = surface_data.loc[surface_data["Tournament"] == self.current_tournament, "Surface Speed"]
        if current_surface_speed.empty:
            current_surface_speed = None
        else:
            current_surface_speed = float(current_surface_speed.iloc[0])
        
        return spw / current_surface_speed, rpw * current_surface_speed
    

In [146]:
draper_stats = PlayerServeReturnStats("Jack", "Draper", 8, "Indian Wells Masters")

In [147]:
draper_spw, draper_rpw = draper_stats.estimate_spw_rpw()
print("SPW: ", draper_spw)
print("RPW: ", draper_rpw)

SPW:  79.10035060143143
RPW:  34.016920000000006


In [148]:
shelton_stats = PlayerServeReturnStats("Ben", "Shelton", 8, "Indian Wells Masters")

In [149]:
shelton_spw, shelton_rpw = shelton_stats.estimate_spw_rpw()
print("SPW: ", shelton_spw)
print("RPW: ", shelton_rpw)

SPW:  81.23448061382219
RPW:  28.440659999999994


In [150]:
draper_v_shelton_spw = (draper_spw + (100 - shelton_rpw)) / 2
shelton_v_draper_spw = (shelton_spw + (100 - draper_rpw)) / 2
print("Draper SPW: ", draper_v_shelton_spw)
print("Shelton SPW: ", shelton_v_draper_spw)

Draper SPW:  75.32984530071572
Shelton SPW:  73.60878030691109


### Markov Chain 

#### Markov Code

In [132]:
game_states = ['0-0', '15-0', '0-15', '30-0', '15-15', '0-30', '40-0', '30-15', '15-30','0-40', 
               '40-15', '30-30', '15-40', '40-30', '30-40', 'Deuce', 'Advantage Player 1',
               'Advantage Player 2', 'Game Player 1', 'Game Player 2']

set_states = ["0-0", "1-0", "0-1", "2-0", "1-1", "0-2", "3-0", "2-1", "1-2", "0-3", "4-0", "3-1", 
              "2-2", "1-3", "0-4", "5-0", "4-1", "3-2", "2-3", "1-4",  "0-5", "5-1", "4-2", "3-3", 
              "2-4", "1-5", "5-2", "4-3", "3-4", "2-5", "5-3", "4-4", "3-5", "5-4", "4-5", "5-5", 
              "6-5", "5-6", "Set Player 1", "Set Player 2", "6-6"]

tiebreak_states = ["0-0", "1-0", "0-1", "2-0", "1-1", "0-2", "3-0", "2-1", "1-2", "0-3",
                   "4-0", "3-1", "2-2", "1-3", "0-4", "5-0", "4-1", "3-2", "2-3", "1-4",
                   "0-5", "6-0", "5-1", "4-2", "3-3", "2-4", "1-5", "0-6", "6-1", "5-2",
                   "4-3", "3-4", "2-5", "1-6", "6-2", "5-3", "4-4", "3-5", "2-6", "6-3",
                   "5-4", "4-5", "3-6", "6-4", "5-5", "4-6", "6-5", "5-6", "6-6", 
                   "Ad Player 1", "Ad Player 2", "Game Player 1", "Game Player 2"]

In [133]:
def serve_return_win_perc(first_name, last_name):
    stat_df = filter_results(search_player(first_name, last_name), f"{first_name} {last_name}")
    serve_df = stat_df[['first_in', 'first_win_perc', 'second_win_perc']].dropna()
    serve_df['service_point_win_perc'] = (serve_df['first_in'] * serve_df['first_win_perc']) + ((1 - serve_df['first_in']) * serve_df['second_win_perc'])
    avg_service_point_win_perc = serve_df['service_point_win_perc'].mean()
    
    return_df = stat_df[['opp_first_in', 'first_return_win_perc' ,'second_return_win_perc']].dropna()
    return_df['return_point_win_perc'] = (return_df['opp_first_in'] * return_df['first_return_win_perc']) + ((1 - return_df['opp_first_in']) * return_df['second_return_win_perc'])
    avg_return_point_win_perc = return_df['return_point_win_perc'].mean()
    
    return avg_service_point_win_perc, avg_return_point_win_perc


In [134]:
serve_pc, return_pc = serve_return_win_perc("Alexander", "Zverev")

print(serve_pc)
print(return_pc)

0.7126916777606077
0.3739518339745773


In [135]:
def get_fund_matrix(transition_mat, num_absorbing):
    """
    Get the fundamental matrix for an absorbing markov chain
    
    :param transition_mat: the transition matrix of the absorbing markov chain
    :param num_absorbing: the number of absorbing states in the markov chain
    :return: the fundamental matrix of the absoribing markov chain
    """
    Q_mat = transition_mat[:-num_absorbing, :-num_absorbing]
    R_mat = transition_mat[:-num_absorbing, -num_absorbing:]
    identity_mat = np.eye(len(transition_mat) - num_absorbing)
    N_mat = np.linalg.inv(identity_mat - Q_mat)
    fund_mat = np.dot(N_mat, R_mat)
    return fund_mat

In [136]:
def get_game_transition_matrix(service_pc):
    """
    Create the transition matrix with states:
    ['0-0', '15-0', '0-15', '30-0', '15-15', '0-30', '40-0', '30-15', '
    15-30','0-40', '40-15', '30-30', '15-40', '40-30', '30-40', 'Deuce', '
    Advantage Player 1', 'Advantage Player 2', 'Game Player 1', 'Game Player 2']
    
    :param service_pc: the decimal percentage that a player wins their serve
    :return: transition matrix for a game of tennis
    """
    transition_matrix = np.zeros((20, 20))
    transition_matrix[19][19] = 1
    transition_matrix[18][18] = 1
    return_pc = 1 - service_pc
    service_coords = [(0, 1), (1, 3), (2, 4), (3, 6), (4, 7), (5, 8), (6, 18), (7, 10), (8, 11), (9, 12), (10, 18), (11, 13), (12, 14), (13, 18), (14, 15), (15, 16), (16, 18), (17, 15)]
    return_coords = [(0, 2), (1, 4), (2, 5), (3, 7), (4, 8), (5, 9), (6, 10), (7, 11), (8, 12), (9, 19), (10, 13), (11, 14), (12, 19), (13, 15), (14, 19), (15, 17), (16, 15), (17, 19)]
    for (i, j) , (x, y) in zip(service_coords, return_coords):
        transition_matrix[i, j] = service_pc
        transition_matrix[x, y] = return_pc
    return transition_matrix

In [137]:
def write_tiebreak_transition_matrix():
    """
    Get the coordinates of transition probabilities for a tennis tiebreak with
    states outlined in the in the items list (excluding absorbing states
    Game Player 1, Game Player2)
    
    :return: the coordinates for the transition probabilities for each players serve and return
    """
    items = ["0-0", "1-0", "0-1", "2-0", "1-1", "0-2", "3-0", "2-1", "1-2", "0-3",
             "4-0", "3-1", "2-2", "1-3", "0-4", "5-0", "4-1", "3-2", "2-3", "1-4",
             "0-5", "6-0", "5-1", "4-2", "3-3", "2-4", "1-5", "0-6", "6-1", "5-2",
             "4-3", "3-4", "2-5", "1-6", "6-2", "5-3", "4-4", "3-5", "2-6", "6-3",
             "5-4", "4-5", "3-6", "6-4", "5-5", "4-6", "6-5", "5-6", "6-6", 
             "Ad Player 1", "Ad Player 2"]
    item_index_dict = {item: index for index, item in enumerate(items)}
    p1_serve_coords = []
    p2_return_coords = []
    p2_serve_coords = []
    p1_return_coords = []
    server = True
    
    for idx, item in enumerate(items):
        if item[0].isdigit():
            p1_points = int(item[0])
            p2_points = int(item[-1])
            next_item = items[idx + 1]
            if next_item[0].isdigit() and idx < len(items) - 2:
                next_point_sum = int(next_item[0]) + int(next_item[-1])
            if p1_points + 1 < 7 and p2_points + 1 < 7:
                p1_win_coords = (idx, item_index_dict[f"{p1_points + 1}-{p2_points}"])
                p2_win_coords = (idx, item_index_dict[f"{p1_points}-{p2_points + 1}"])
            elif p1_points + 1 == 7 and p2_points + 1 < 7:
                p1_win_coords = (idx, 51)
                p2_win_coords = (idx, item_index_dict[f"{p1_points}-{p2_points + 1}"])
            elif p1_points + 1 < 7 and p2_points + 1 == 7:
                p1_win_coords = (idx, item_index_dict[f"{p1_points + 1}-{p2_points}"])
                p2_win_coords = (idx, 52)
            elif p1_points + 1 == 7 and p2_points + 1 == 7:
                p1_win_coords = (idx, 49)
                p2_win_coords = (idx, 50)
        else:
            if item[-1] == "1":
                p1_win_coords = (idx, 51)
                p2_win_coords = (idx, 44)
            elif item[-1] == "2":
                p1_win_coords = (idx, 44)
                p2_win_coords = (idx, 52)
        if server:
            p1_serve_coords.append(p1_win_coords)
            p2_return_coords.append(p2_win_coords)
            if next_item[0] == "A" or next_point_sum % 4 == 1 :
                server = False
        elif not server:
            p2_serve_coords.append(p2_win_coords)
            p1_return_coords.append(p1_win_coords)
            if next_point_sum % 4 == 3:
                server = True
            
            
    return p1_serve_coords, p2_return_coords, p2_serve_coords, p1_return_coords

In [138]:
def get_tiebreak_transition_matrix(p1_service_perc, p2_service_perc):
    """
    Create the transition matrix with states:
    ["0-0", "1-0", "0-1", "2-0", "1-1", "0-2", "3-0", "2-1", "1-2", "0-3",
    "4-0", "3-1", "2-2", "1-3", "0-4", "5-0", "4-1", "3-2", "2-3", "1-4",
    "0-5", "6-0", "5-1", "4-2", "3-3", "2-4", "1-5", "0-6", "6-1", "5-2",
    "4-3", "3-4", "2-5", "1-6", "6-2", "5-3", "4-4", "3-5", "2-6", "6-3",
    "5-4", "4-5", "3-6", "6-4", "5-5", "4-6", "6-5", "5-6", "6-6", 
    "Ad Player 1", "Ad Player 2", "Game Player 1", "Game Player 2"]
    
    :param p1_service_pc: the decimal percentage that a player 1 wins their serve
    :param p1_service_pc: the decimal percentage that a player 2 wins their serve
    :return: transition matrix for a tiebreak in a tennis match
    """
    
    transition_matrix = np.zeros((53, 53))
    transition_matrix[52][52] = 1
    transition_matrix[51][51] = 1
    
    p1_serve_coords, p2_return_coords, p2_serve_coords, p1_return_coords = write_tiebreak_transition_matrix()
    
    p1_return_perc = 1 - p2_service_perc
    p2_return_perc = 1 - p1_service_perc
    
    for t in range(0, len(p2_serve_coords)):
        if t < 24:
            n, m = p1_serve_coords[t]
            o, p = p2_return_coords[t]
        i, j = p2_serve_coords[t]
        x, y = p1_return_coords[t]
        
        transition_matrix[i, j] = p2_service_perc
        transition_matrix[x, y] = p1_return_perc
        transition_matrix[n, m] = p1_service_perc
        transition_matrix[o, p] = p2_return_perc
    
    return transition_matrix

In [139]:
def get_set_transition_matrix(p1_service_game_perc, p2_service_game_perc):
    """
    Create the transition matrix with states:
    ["0-0", "1-0", "0-1", "2-0", "1-1", "0-2", "3-0", "2-1", "1-2", "0-3", "4-0", "3-1", 
    "2-2", "1-3", "0-4", "5-0", "4-1", "3-2", "2-3", "1-4",  "0-5", "5-1", "4-2", "3-3", 
    "2-4", "1-5", "5-2", "4-3", "3-4", "2-5", "5-3", "4-4", "3-5", "5-4", "4-5", "5-5", 
    "6-5", "5-6", "Set Player 1", "Set Player 2", "6-6"]
    
    :param p1_service_game_perc: the decimal percentage that a player 1 wins a game
    :param p2_service_game_perc: the decimal percentage that a player 2 wins a game
    :return: transition matrix for a set in a tennis match
    """
    transition_matrix = np.zeros((41, 41))
    transition_matrix[40][40] = 1
    transition_matrix[39][39] = 1
    transition_matrix[38][38] = 1
    
    p1_return_game_perc = 1 - p2_service_game_perc
    p2_return_game_perc = 1 - p1_service_game_perc
    
    p1_serve_coords = [(0, 1), (3, 6), (4, 7), (5, 8), (10, 15), (11, 16), (12, 17), (13, 18), (14, 19), (21, 38), (22, 26), (23, 27), (24, 28), (25, 29), (30, 38), (31, 33), (32, 34), (35, 36)]
    p2_return_coords = [(0, 2), (3, 7), (4, 8), (5, 9), (10, 16), (11, 17), (12, 18), (13, 19), (14, 20), (21, 26), (22, 27), (23, 28), (24, 29), (25, 39), (30, 33), (31, 34), (32, 39), (35, 37)]
    
    p2_serve_coords = [(1, 4), (2, 5), (6, 10), (7, 11), (8, 12), (9, 13), (15, 21), (16, 22), (17, 23), (18, 24), (19, 25), (20, 39), (26, 30), (27, 31), (28, 32), (29, 39), (33, 35), (34, 39), (36, 40), (37, 39)]
    p1_return_coords = [(1, 3), (2, 4), (6, 11), (7, 12), (8, 13), (9, 14), (15, 38), (16, 21), (17, 22), (18, 23), (19, 24), (20, 25), (26, 38), (27, 30), (28, 31), (29, 32), (33, 38), (34, 35), (36, 38), (37, 40)]
    
    for t in range(0, len(p1_return_coords)):
        if t < 18:
            i, j = p1_serve_coords[t]
            x, y = p2_return_coords[t]
        n, m = p2_serve_coords[t]
        o, p = p1_return_coords[t]
        
        transition_matrix[i, j] = p1_service_game_perc
        transition_matrix[x, y] = p2_return_game_perc
        transition_matrix[n, m] = p2_service_game_perc
        transition_matrix[o, p] = p1_return_game_perc
    
    return transition_matrix

In [140]:
def get_set_win_perc(p1_service_perc, p2_service_perc):
    """
    Get the percentage chance of each of 2 players to win a set of tennis
    given their service point win percentages
    
    :param p1_service_perc: the decimal percentage winrate of player 1 on their serve
    :param p2_service_perc: the decimal percentage winrate of player 2 on their serve
    :return: win percentages for a set of tennis for each player
    """
    p1_game_transition_matrix = get_game_transition_matrix(p1_service_perc)
    p2_game_transition_matrix = get_game_transition_matrix(p2_service_perc)
    
    p1_fund_matrix = get_fund_matrix(p1_game_transition_matrix, 2)
    p2_fund_matrix = get_fund_matrix(p2_game_transition_matrix, 2)
    
    p1_service_game_perc = p1_fund_matrix[0][0]
    p2_service_game_perc = p2_fund_matrix[0][0]
    
    set_transition_matrix = get_set_transition_matrix(p1_service_game_perc, p2_service_game_perc)
    set_fund_matrix = get_fund_matrix(set_transition_matrix, 3)
    tiebreak_perc = set_fund_matrix[0][-1]
    tiebreak_transition_matrix = get_tiebreak_transition_matrix(p1_service_perc, p2_service_perc)
    tiebreak_fund_matrix = get_fund_matrix(tiebreak_transition_matrix, 2)
    p1_tiebreak_winner = tiebreak_fund_matrix[0][0]
    p2_tiebreak_winner = tiebreak_fund_matrix[0][1]

    p1_win_perc = set_fund_matrix[0][0] + tiebreak_perc * p1_tiebreak_winner
    p2_win_perc = set_fund_matrix[0][1] + tiebreak_perc * p2_tiebreak_winner
    
    return p1_win_perc, p2_win_perc

In [141]:
def get_match_prob(p1_service_perc, p2_service_perc, write_mat=False):
    """
    Get the percentage chance of each of 2 players to win a matfch of tennis 
    given their service point win percentages and how many games are in the match
    
    :param p1_service_perc: the decimal percentage winrate of player 1 on their serve
    :param p2_service_perc: the decimal percentage winrate of player 2 on their serve
    :return: win percentages for a match of tennis for each player
    """
    p1_set_win_p1_serves_first, p2_set_win_p1_serves_first = get_set_win_perc(p1_service_perc, p2_service_perc, write_mat=write_mat)
    p2_set_win_p2_serves_first, p1_set_win_p2_serves_first = get_set_win_perc(p2_service_perc, p1_service_perc)
    
    p1_win_serving_first = (
        # Win in straight sets
        p1_set_win_p1_serves_first * p1_set_win_p2_serves_first +
        # Win in three sets
        p1_set_win_p1_serves_first * p2_set_win_p2_serves_first * p1_set_win_p1_serves_first +
        p2_set_win_p1_serves_first * p1_set_win_p2_serves_first * p1_set_win_p1_serves_first)
    
    # Player 2 serving first scenarios
    p1_win_serving_second = (
        # Win in straight sets
        p1_set_win_p2_serves_first * p1_set_win_p1_serves_first +
        # Win in three sets
        p1_set_win_p2_serves_first * p2_set_win_p1_serves_first * p1_set_win_p2_serves_first +
        p2_set_win_p2_serves_first * p1_set_win_p1_serves_first * p1_set_win_p2_serves_first)
    
    p2_win_serving_first = (
            # Win in straight sets
            p2_set_win_p2_serves_first * p2_set_win_p1_serves_first +
            # Win in three sets
            p2_set_win_p2_serves_first * p1_set_win_p1_serves_first * p2_set_win_p2_serves_first +
            p1_set_win_p2_serves_first * p2_set_win_p1_serves_first * p2_set_win_p2_serves_first)
        
    # Player 2's scenarios when serving second
    p2_win_serving_second = (
        # Win in straight sets
        p2_set_win_p1_serves_first * p2_set_win_p2_serves_first +
        # Win in three sets
        p2_set_win_p1_serves_first * p1_set_win_p2_serves_first * p2_set_win_p1_serves_first +
        p1_set_win_p1_serves_first * p2_set_win_p2_serves_first * p2_set_win_p1_serves_first)
    
    p1_win_prob = (p1_win_serving_first + p1_win_serving_second) / 2
    p2_win_prob = (p2_win_serving_first + p2_win_serving_second) / 2
    
    return p1_win_prob, p2_win_prob


In [142]:
def get_match_prob(p1_service_perc, p2_service_perc):
    """
    Get the percentage chance of each of 2 players to win a matfch of tennis 
    given their service point win percentages and how many games are in the match
    
    :param p1_service_perc: the decimal percentage winrate of player 1 on their serve
    :param p2_service_perc: the decimal percentage winrate of player 2 on their serve
    :return: win percentages for a match of tennis for each player
    """
    p1_set_win_p1_serves_first, p2_set_win_p1_serves_first = get_set_win_perc(p1_service_perc, p2_service_perc)
    p2_set_win_p2_serves_first, p1_set_win_p2_serves_first = get_set_win_perc(p2_service_perc, p1_service_perc)
    
    p1_win_serving_first = (
        # Win in straight sets
        p1_set_win_p1_serves_first * p1_set_win_p2_serves_first +
        # Win in three sets
        p1_set_win_p1_serves_first * p2_set_win_p2_serves_first * p1_set_win_p1_serves_first +
        p2_set_win_p1_serves_first * p1_set_win_p2_serves_first * p1_set_win_p1_serves_first)
    
    # Player 2 serving first scenarios
    p1_win_serving_second = (
        # Win in straight sets
        p1_set_win_p2_serves_first * p1_set_win_p1_serves_first +
        # Win in three sets
        p1_set_win_p2_serves_first * p2_set_win_p1_serves_first * p1_set_win_p2_serves_first +
        p2_set_win_p2_serves_first * p1_set_win_p1_serves_first * p1_set_win_p2_serves_first)
    
    p2_win_serving_first = (
            # Win in straight sets
            p2_set_win_p2_serves_first * p2_set_win_p1_serves_first +
            # Win in three sets
            p2_set_win_p2_serves_first * p1_set_win_p1_serves_first * p2_set_win_p2_serves_first +
            p1_set_win_p2_serves_first * p2_set_win_p1_serves_first * p2_set_win_p2_serves_first)
        
    # Player 2's scenarios when serving second
    p2_win_serving_second = (
        # Win in straight sets
        p2_set_win_p1_serves_first * p2_set_win_p2_serves_first +
        # Win in three sets
        p2_set_win_p1_serves_first * p1_set_win_p2_serves_first * p2_set_win_p1_serves_first +
        p1_set_win_p1_serves_first * p2_set_win_p2_serves_first * p2_set_win_p1_serves_first)
    
    p1_win_prob = (p1_win_serving_first + p1_win_serving_second) / 2
    p2_win_prob = (p2_win_serving_first + p2_win_serving_second) / 2
    
    return p1_win_prob, p2_win_prob


#### Run Prediction

In [152]:
def predict_match(player1_first, player1_last, player2_first, player2_last, current_tournament, num_weeks=-1):
    player1_stats = PlayerServeReturnStats(player1_first, player1_last, num_weeks, current_tournament)
    player2_stats = PlayerServeReturnStats(player2_first, player2_last, num_weeks, current_tournament)
    
    player1_spw, player1_rpw = player1_stats.estimate_spw_rpw()
    player2_spw, player2_rpw = player2_stats.estimate_spw_rpw()
    
    player_1_combined_spw = (player1_spw + (100 - player2_rpw)) / 2
    player_2_combined_spw = (player2_spw + (100 - player1_rpw)) / 2
    
    return get_match_prob(player_1_combined_spw / 100, player_2_combined_spw / 100)

In [162]:
p1_first = "Jannik"
p1_last = "Sinner"

p2_first = "Daniil"
p2_last = "Medvedev"

current_tournament = "Indian Wells Masters"

p1_win_perc, p2_win_perc = predict_match(p1_first, p1_last, p2_first, p2_last, current_tournament, num_weeks=8)

print("The probability of player 1 winning the match is: ", p1_win_perc)
print("The probability of player 2 winning the match is: ", p2_win_perc)

The probability of player 1 winning the match is:  0.5322194866562717
The probability of player 2 winning the match is:  0.46778051334372817


In [161]:
### Need to add in a way to weight by opponent ranking
### Need to add a way to average winning percentage stats by the number of games played in the match

# Additional Features

## Backtesting

### Setting up a Framework 

1. Research how to set up a backtesting framework, what apis, libraries already exist for it
2. Go by the predicted service percentages as well as how winning percentages translate to implied odds
3. Set up a method for calculating an implied return on a tournament, on a player's recent X matches, or all matches in the previous X days

### Feature Backtesting

1. 

## Changing Weights By Additional Features

1. Opponent Rankings
2. Recent Results
3. Head to head

### Opponent Rankings

1. How to group rankings in order to weight final percentage - by range or difference?
2. Should this be taken into account only when looking at their historical data or at the player ranking difference as well?    


#### Possible Approaches
1. Get the ranking difference for each matchup in their historical data, do some research on how to scale this towards the actual percentage differences  
2. Could do this by comparing predicted service and return win percentages with reality in previous matchups through a linear regression or another machine learning model  
3. 

### Recent Results

1. How much to weight recent results by vs historical form
2. Perform linear regression/random forest to do this?  

## Player Attributes  

1. Lefty vs Righty
2. Height
3. Playstyle